In [1]:
import pyTigerGraph as tg

conn = tg.TigerGraphConnection("http://3.22.188.182", graphname="KDD_2022_NFT")

In [2]:
f = conn.gds.featurizer()

f.installAlgorithm("tg_fastRP")

'tg_fastRP'

In [3]:
params = {"v_type": ["Category", "NFT_Collection", "NFT"], 
          "e_type": ["COLLECTION_HAS_NFT", "CATEGORY_HAS_NFT", "NFT_IN_CATEGORY", "NFT_IN_COLLECTION"], 
          "weights": "1,2,4", 
          "beta": -0.1,
          "k": 3,
          "reduced_dim": 64, 
          "sampling_constant": 3,
          "random_seed": 42,
          "print_accum": False,
          "result_attr": "fastrp_embedding"}

f.runAlgorithm("tg_fastRP", params)

[]

In [4]:
ExprFunctions=""  # For enterprise users, please use the link you received.
ExprUtil=""  # For enterprise users, please use the link you received.
#conn.installUDF(ExprFunctions, ExprUtil)

In [5]:
conn.gds.configureKafka(kafka_address="kaf.gke.tigergraphlabs.com:19092")

In [6]:
train_loader = conn.gds.neighborLoader(
    v_in_feats={"Transaction": ["seller_k_size", "buyer_k_size"], 
                "NFT_User": ["pagerank", "kcore_size"], 
                "NFT": ["fastrp_embedding"], 
                "NFT_Collection": ["fastrp_embedding"], 
                "Category": ["fastrp_embedding"]},
    v_out_labels={"Transaction": ["usd_price"]},
    v_extra_feats={"Transaction":  ["train"]},
    filter_by={"Transaction": "train"},
    shuffle=True,
    batch_size=512,
    buffer_size=4,
    add_self_loop=True,
    reverse_edge=True
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.


In [7]:
for batch in train_loader:
    print(batch.metadata())
    break

/opt/anaconda3/envs/kdd2022/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


(['Transaction', 'NFT', 'NFT_User', 'NFT_Collection', 'Category'], [('Transaction', 'NFT_SOLD_BY', 'NFT_User'), ('Transaction', 'NFT_BOUGHT_BY', 'NFT_User'), ('Transaction', 'FOR_SALE_OF', 'NFT'), ('NFT', 'HAD_TRANSACTION', 'Transaction'), ('NFT', 'NFT_IN_COLLECTION', 'NFT_Collection'), ('NFT', 'NFT_IN_CATEGORY', 'Category'), ('NFT_User', 'USER_BOUGHT_FROM', 'NFT_User'), ('NFT_User', 'USER_BOUGHT_NFT', 'Transaction'), ('NFT_User', 'USER_SOLD_NFT', 'Transaction'), ('NFT_User', 'USER_SOLD_TO', 'NFT_User')])


In [8]:
train_loader.num_batches

93

In [9]:
import torch
import torch.nn.functional as F
from torch_geometric.nn import GATConv, to_hetero


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Create a normal (homogeneous) GAT model
class GAT(torch.nn.Module):
    def __init__(
        self, num_layers, out_dim, dropout, hidden_dim, num_heads
    ):
        super().__init__()
        self.dropout = dropout
        self.layers = torch.nn.ModuleList()
        for i in range(num_layers):
            in_units = (-1, -1) if i == 0 else hidden_dim * num_heads
            out_units = out_dim if i == (num_layers - 1) else hidden_dim
            heads = 1 if i == (num_layers - 1) else num_heads
            self.layers.append(
                GATConv(in_units, out_units, heads=heads, dropout=dropout)
            )
        self.double()

    def reset_parameters(self):
        for layer in self.layers:
            layer.reset_parameters()

    def forward(self, x, edge_index):
        x = x.float()
        for layer in self.layers[:-1]:
            x = layer(x, edge_index)
            x = F.elu(x)
            x = F.dropout(x, p=self.dropout, training=self.training)
        x = self.layers[-1](x, edge_index)
        return x
    
model = GAT(
    num_layers=2,
    out_dim=1,
    dropout=0.7,
    hidden_dim=8,
    num_heads=1,
)

# Convert it to a heterogeneous model. See https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.to_hetero_transformer.to_hetero for details.
model = to_hetero(model, batch.metadata(), aggr='mul').to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

mae = torch.nn.L1Loss()

In [10]:
for i in range(5):
    epochLoss = 0
    epochMae = 0
    j = 0
    for batch in train_loader:
        model.train()
        optimizer.zero_grad()
        out = model(batch.x_dict, batch.edge_index_dict)
        mask = batch["Transaction"].train
        loss = F.smooth_l1_loss(out["Transaction"][mask].flatten(), batch["Transaction"].y[mask])
        loss.backward()
        optimizer.step()
        epochLoss += loss.item()
        batchMae = mae(out["Transaction"][mask].flatten(), batch["Transaction"].y[mask])
        epochMae += batchMae.item()
        #print("Batch:", j, "Loss:", loss.item(), "MAE:", batchMae.item())
        j += 1
    print("EPOCH:", i, "LOSS:", epochLoss / train_loader.num_batches, "MAE:", epochMae / train_loader.num_batches)

EPOCH: 0 LOSS: 71.24119676679322 MAE: 71.59755258967495
EPOCH: 1 LOSS: 71.83530232614262 MAE: 72.19175388601833
EPOCH: 2 LOSS: 72.14543973863248 MAE: 72.50141460873414
EPOCH: 3 LOSS: 71.43679730251202 MAE: 71.79342527898513
EPOCH: 4 LOSS: 73.55483734309252 MAE: 73.91114001398968


In [11]:
test_loader = conn.gds.neighborLoader(
    v_in_feats={"Transaction": ["seller_k_size", "buyer_k_size"], 
                "NFT_User": ["pagerank", "kcore_size"], 
                "NFT": ["fastrp_embedding"], 
                "NFT_Collection": ["fastrp_embedding"], 
                "Category": ["fastrp_embedding"]},
    v_out_labels={"Transaction": ["usd_price"]},
    v_extra_feats={"Transaction":  ["test"]},
    filter_by={"Transaction": "test"},
    shuffle=False,
    batch_size=2048,
    add_self_loop=True,
    reverse_edge=True
)

Installing and optimizing queries. It might take a minute if this is the first time you use this loader.
Query installation finished.


In [12]:
totLoss = 0
totMAE = 0
for batch in test_loader:
    model.eval()
    with torch.no_grad():
        out = model(batch.x_dict, batch.edge_index_dict)
        mask = batch["Transaction"].test
        loss = F.mse_loss(out["Transaction"][mask].flatten(), batch["Transaction"].y[mask])
    epochLoss += loss.item()
    totMAE = mae(out["Transaction"][mask].flatten(), batch["Transaction"].y[mask])
    totLoss += batchMae.item()
print("LOSS:", epochLoss / test_loader.num_batches, "MAE:", epochMae / test_loader.num_batches)

LOSS: 4089096.551066067 MAE: 1145.62267021684
